In [ ]:
import sys
import os
import numpy as np

In [ ]:
from DiffusionAnalysis.loaders import DatDirectoryStructureLoader
from DiffusionAnalysis.trajectory import DisplacementTrajectory
from DiffusionAnalysis.analysis import MSDAnalysis


In [ ]:
loader = DatDirectoryStructureLoader('dat_directory', structures_slice=slice(0,1000))

In [ ]:
# traj = DisplacementTrajectory(loader, timestep=0.1, time_unit='ps' )

In [ ]:
# traj.generate_displacement_trajectory(host_atoms=['H'], show_progress=True)
# # #save as a pickle (not implemented in the class)
import pickle
# with open('displacement_traj.pickle', 'wb') as f:
#     pickle.dump(traj, f)

#load the pickle
with open('displacement_traj.pickle', 'rb') as f:
    traj = pickle.load(f)
    


In [ ]:
print(traj.displacement_trajectory[0,:,:])

#sum each timestep to get total displacement
total_displacement = traj.displacement_trajectory.sum(axis=1)
print(total_displacement[0])

#print the traj.displacement_trajectory nbytes in mb
print(traj.displacement_trajectory.nbytes/1e6)
print(traj._check_memory_usage())


In [ ]:
loader.reset()

atoms0 = next(loader)
for i in range(8):
    atoms2 = next(loader)

atoms10 = next(loader)

print(atoms0.get_scaled_positions()[0])
print(atoms10.get_scaled_positions()[0])

print(atoms10.get_scaled_positions()[0]- atoms0.get_scaled_positions()[0] )

print(traj.displacement_trajectory.shape)

In [ ]:
msd_analysis = MSDAnalysis(traj)

# Assuming you have an instance of MSDAnalysis called 'msd_analysis'

# Calculate the 3D MSD
msd_3d = msd_analysis.calculate_msd(return_3d_msd=True)
fig1 = msd_analysis.plot_msd((msd_3d), title='MSD Plot', legend_loc='upper left', skip_points=4)
fig1_log = msd_analysis.plot_msd((msd_3d), title='MSD Plot (Log Scale)', legend_loc='upper left', skip_points=4, log_scale=True)


# Calculate the MSD for each direction (x, y, z)
msd_x, msd_y, msd_z = msd_analysis.calculate_msd(return_3d_msd=False)

# Plot the MSD for each direction
fig2 = msd_analysis.plot_msd((msd_x, msd_y, msd_z), labels=['x','y' ,'z'], title='MSD Plot', legend_loc='upper left', skip_points=4)
fig2.show()
# Calculate and plot the MSD along a non-normalized lattice vector for host atoms, correcting for framework drift
lattice_vector = np.array([2, 0, 0])  # Example non-normalized lattice vector
msd_along_lattice = msd_analysis.calculate_msd(lattice_vector = lattice_vector)
fig3 = msd_analysis.plot_msd(msd_along_lattice, labels=f'MSD Along Lattice Vector {lattice_vector}', title='MSD Along Lattice Vector (Drift Uncorrected)', skip_points=4)
fig3.show()


In [ ]:
from DiffusionAnalysis.analysis import VanHoveAnalysis
import numpy as np
import matplotlib.pyplot as plt


# Create a VanHoveAnalysis object
van_hove_analyzer = VanHoveAnalysis(traj)

# Set the parameters for the analysis
tau_values = [1, 5, 10, 100, 500]  # Time lags in number of timesteps
r_range = (0, 10)  # Range of distances to consider
n_bins = 100  # Number of bins for the histogram

self_data = []
for tau in tau_values:
    bin_centers, hist = van_hove_analyzer.calculate_van_hove(tau, r_range, n_bins,
                                                             mode='self', )
    self_data.append((bin_centers, hist, tau))

# Plot the self-part of the van Hove correlation function
self_fig = van_hove_analyzer.plot_van_hove(self_data, mode='self', title='Self-part of the Van Hove Correlation Function')
plt.show()

distinct_data = []
for tau in tau_values:
    bin_centers_distinct, hist_distinct = van_hove_analyzer.calculate_van_hove(tau, r_range, n_bins, mode='distinct')
    distinct_data.append((bin_centers_distinct, hist_distinct, tau))

# Plot the distinct-part of the van Hove correlation function
distinct_fig = van_hove_analyzer.plot_van_hove(distinct_data, mode='distinct', title='Distinct-part of the Van Hove Correlation Function')
plt.show()


In [ ]:
from DiffusionAnalysis.analysis import tMSDAnalysis

# Create an instance of the tMSDAnalysis class
tmsd_analysis = tMSDAnalysis(traj)

# Define the minimum and maximum time lag values (in steps) and the number of points
min_tau = 1
max_tau = 1000
num_points = 100

# Calculate tMSD
time_lag_values, tMSD_values = tmsd_analysis.calculate_tMSD(min_tau, max_tau, num_points)

# Plot the tMSD data
temperature = 500  # Example temperature value
label = f'{temperature} K'
fig = tmsd_analysis.plot_tMSD(time_lag_values, tMSD_values, label)

fig2 = tmsd_analysis.plot_tMSD_exponent(time_lag_values, tMSD_values, label, average_window_size=3) 




In [ ]:
from matplotlib import pyplot as plt

# Set the parameters
time_lags = [1, 10, 100]  # Example time lags
num_bins = 100
lattice_vector = np.array([1, 0, 0])  # Example lattice vector
time_unit = traj.time_unit.value
time_step = traj.timestep

# Create a figure with two subplots
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

# Plot the probability distribution along the lattice vector
for time_lag in time_lags:
    # Calculate the displacement distribution along the lattice vector
    bin_centers, hist = msd_analysis.calculate_displacement_distribution(time_lag=time_lag, num_bins=num_bins,
                                                                         use_3d=False, lattice_vector=lattice_vector)

    # Plot the distribution
    ax1.plot(bin_centers, hist, label=f"Time lag: {time_lag*time_step} ({time_unit})")

# Set the axis labels and title for the lattice vector plot
ax1.set_xlabel(f"Displacement along Lattice Vector")
ax1.set_ylabel(r"Probability")
ax1.set_title("Probability Distribution of Displacement along Lattice Vector")
ax1.legend(loc='best')
ax1.set_xlim(-1.5,1.5)

# Plot the probability distribution of the total 3D displacement
for time_lag in time_lags:
    # Calculate the displacement distribution in 3D
    bin_centers, hist = msd_analysis.calculate_displacement_distribution(time_lag=time_lag, num_bins=num_bins,
                                                                         use_3d=True)

    # Plot the distribution
    ax2.plot(bin_centers, hist, label=f"Time lag: {time_lag*time_step} ({time_unit})")

# Set the axis labels and title for the 3D displacement plot
ax2.set_xlabel(f"3D Displacement")
ax2.set_ylabel(r"Probability")
ax2.set_title("Probability Distribution of Total 3D Displacement")
ax2.legend(loc='best')
ax2.set_xlim(0,3)


# Adjust the spacing between subplots
plt.tight_layout()

# Display the plot
plt.show()